# Spark Session

* This is the entry point to Spark SQL. It is one of the very first objects you create while developing a Spark SQL application.
* As a Spark developer, you create a SparkSession using the SparkSession.builder method (that gives you access to Builder API that you use to configure the session).

In [1]:
# pip install delta

In [2]:
# pip install delta-spark

In [3]:
# Import Necessary Libraries
import os
from delta.tables import *
from delta.tables import DeltaTable
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,array,ArrayType,DateType,TimestampType, FloatType
from pyspark.sql import functions as f
from pyspark.sql.functions import udf
import hashlib
import datetime
import urllib.request
import json
from datetime import timedelta, date
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SQLContext
from itertools import islice
from pyspark.sql.functions import col
import sys

In [4]:
# Builder API
# Spark session & context
spark=SparkSession.builder.master("local").appName("Hive-Test").enableHiveSupport().getOrCreate()


# Get configurations
# configurations = spark.sparkContext.getConf().getAll()
# for item in configurations: print(item)

In [5]:
# Read CustomersData From Minio
customers = spark.read.option("header",True).csv("s3a://bronze/sales/customers/2022/07/02/09/customers.csv")

# Show Top 5 
customers.show(5)

+-----------+--------------+--------------------+-----------+----------+--------------+----------------+--------------------+----------------+--------------------+
|customer_id| customer_name|             address|       city|postalcode|       country|           phone|               email|     credit_card|          updated_at|
+-----------+--------------+--------------------+-----------+----------+--------------+----------------+--------------------+----------------+--------------------+
|          1|    Ariel Hale|Ap #660-3260 Pell...|    College|     98362| United States|  1-973-833-9836|amet.metus@Nullat...|5124442517412973|2022-07-21 09:14:...|
|          2| Aubrey Norris|Ap #943-1347 Impe...| Coldstream|   D10 5JV|United Kingdom|    07672 321093|sollicitudin@enim...|5103696625359419|2022-07-21 09:14:...|
|          3|  Bruno Hebert|    8566 Nisi Avenue| Llangollen|   CE2 4WW|United Kingdom|    02794 010514|Donec.non@dapibus...|5132188470727440|2022-07-21 09:14:...|
|          4|   

In [6]:
spark.sql("show databases;").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [10]:
# Read SQL
customers.createOrReplaceTempView("customers")

## Performing Transformations 
uk_customers = spark.sql("""
    SELECT *
    FROM customers
    WHERE country = 'United Kingdom'
    ORDER BY customer_id ASC
    LIMIT 100
""")


# Writing Results to S3
# uk_customers.write.option("header","true").csv("s3a://silver/customers/UKSecret")

In [11]:
# Write Results to Minio in DeltaFormat
uk_customers.write.format("delta").mode("overwrite").option('overwriteSchema','true').save("s3a://silver/delta/customers") 